# Create mask (inc vs dec) based on ML HYCOM data, and create one plot for each event

In [1]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
from datetime import date, timedelta, datetime
import sys
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import netCDF4 as nc4
import glob
import os.path
from mpl_toolkits.axes_grid1 import make_axes_locatable
import webbrowser
import time
import gsw
import matplotlib.colors as mcolors
import scipy.io as scipy
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import matplotlib as mpl
from FUNCTIONS_HYCOM import set_regions, set_regions_tags, find_timedelta64_index, \
create_composite, name_composite, below_above_MLD, plot_simple_pcolor, set_regions_offset, draw_box, \
save_3dvars_as_nc4, save_2dvars_as_nc4, save_1dvars_as_nc4, \
set_cmap, set_clim_plots, set_clim_plots_std, set_clim_plots_diff, set_clim_plots_std_diff, \
function_map_2D_hist, function_plots, function_single_panels_plots, function_single_panels_plots_grey_shading
import pickle as pkl
from holteandtalley import HolteAndTalley
import subprocess
import numpy.matlib

In [2]:
# Open netcdf with both HYCOM and forcing data from all events across all regions, at hourly resolution
combined_regions = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/scripf_for_downloading_forcing_files_CURC/combined_regions_with_rel_hum.nc4')
# combined_regions.depth

In [3]:
# # Specify the new depth levels with a step of 5
# new_depth_levels = np.arange(0, 510, 5)

# # Interpolate along the 'depth' dimension using linear interpolation
# combined_regions_interpolated = combined_regions.interp(depth = new_depth_levels, method='linear')


In [4]:
# REMOVE i_time if it's the same as index_time_start 
i_time = int((np.abs(combined_regions.delta_time - int(-2 * 24 * 60 * 60 * 1E9))).argmin()) # time index where to perform the classification (-2 days)

# Needed for composite calculation & plots
index_time_start = int((np.abs(combined_regions.delta_time - int(-2 * 24 * 60 * 60 * 1E9))).argmin()) # 2 days prior
index_time_end = int((np.abs(combined_regions.delta_time - int(14 * 24 * 60 * 60 * 1E9))).argmin()) # 14 days after
index_time_end_10days = int((np.abs(combined_regions.delta_time - int(10 * 24 * 60 * 60 * 1E9))).argmin()) # 14 days after

# Needed for composite plots
offset_lon = 20
timesteps_before = 2*24 # 2 days prior
timesteps_after = 14*24 # 14 days after
time_plot = np.arange(-timesteps_before,timesteps_after+1,1) * 1/24


In [5]:
# Inputs to decide which incr/decr mask to use
ML_delta = 50 # 1/2 1/3 2/3 1 10 20 30 40 50 60 70 
mode_tag = 'absolute' # 'fraction' # 'absolute'
mask_tag = 'plus50m' # 1ML halfML 1thirdML 2thirdML plus10m plus20m plus30m plus40m plus50m plus60m plus70m

diagonal_plot_tag = False
sanity_check_plot_tag = False
save_combined_file_for_donata = False # True False


In [6]:
# pressure = gsw.p_from_z(-combined_regions_interpolated.depth.values, float(combined_regions_interpolated.latitude.values[0]))
# pressure

In [7]:
# This cell takes around x minutes to run and save the file 
# Calculate MLD using 3 diferent algorithms: temperature, salinity and density 

if save_combined_file_for_donata:
    pressure = gsw.p_from_z(-combined_regions_interpolated.depth.values, float(combined_regions_interpolated.latitude.values[0]))

    # Get dimensions from the existing dataset
    index_dim = combined_regions_interpolated.index.values
    delta_time_dim = combined_regions_interpolated.delta_time.values

    # Create new DataArrays for MLDs
    tempMLD = xr.DataArray(np.nan * np.zeros_like(combined_regions_interpolated['temperature_abs'][:,:,0].values),
                           dims=('index', 'delta_time'),
                           coords={'index': index_dim,
                                   'delta_time': delta_time_dim},
                           name='tempMLD')

    salinityMLD = xr.DataArray(np.nan * np.zeros_like(combined_regions_interpolated['temperature_abs'][:,:,0].values),
                               dims=('index', 'delta_time'),
                               coords={'index': index_dim,
                                       'delta_time': delta_time_dim},
                               name='salinityMLD')

    densityMLD = xr.DataArray(np.nan * np.zeros_like(combined_regions_interpolated['temperature_abs'][:,:,0].values),
                              dims=('index', 'delta_time'),
                              coords={'index': index_dim,
                                      'delta_time': delta_time_dim},
                              name='densityMLD')

    # Add the new DataArrays to the dataset
    combined_regions_interpolated['tempMLD'] = tempMLD
    combined_regions_interpolated['salinityMLD'] = salinityMLD
    combined_regions_interpolated['densityMLD'] = densityMLD


    for i_event in np.arange(combined_regions_interpolated['temperature_abs'].shape[0]):  # Loop across events
        for itime in np.arange(combined_regions_interpolated['temperature_abs'].shape[1]):  # Loop across times

            # Remove NaNs to let the function work
            conservative_temperature_nonans = combined_regions_interpolated.temperature_abs.values[i_event, itime][~np.isnan(combined_regions_interpolated["temperature_abs"].values[i_event, itime])]
            absolute_salinity_nonans = combined_regions_interpolated.salinity_abs.values[i_event, itime][~np.isnan(combined_regions_interpolated["salinity_abs"].values[i_event, itime])]
            potential_density_nonans = combined_regions_interpolated.density_abs.values[i_event, itime][~np.isnan(combined_regions_interpolated["density_abs"].values[i_event, itime])]

            try:
                # Apply function
                h = HolteAndTalley(pressures=pressure[:conservative_temperature_nonans.shape[0]],
                                   temperatures=conservative_temperature_nonans,
                                   salinities=absolute_salinity_nonans,
                                   densities=potential_density_nonans)

                # Save MLD values directly into the dataset
                combined_regions_interpolated["tempMLD"].values[i_event, itime] = h.tempMLD
                combined_regions_interpolated["salinityMLD"].values[i_event, itime] = h.salinityMLD
                combined_regions_interpolated["densityMLD"].values[i_event, itime] = h.densityMLD

            except ValueError as e:
                print(f"Error calculating MLD for event {i_event}, time {itime}: {e}")
                # Handle the error as needed, e.g., set MLD values to a default or NaN
                combined_regions_interpolated["tempMLD"].values[i_event, itime] = np.nan
                combined_regions_interpolated["salinityMLD"].values[i_event, itime] = np.nan
                combined_regions_interpolated["densityMLD"].values[i_event, itime] = np.nan

    pkl.dump(combined_regions_interpolated, open(f'combined_regions_interpolated_with_all_MLD.pkl', 'wb'))


In [8]:
combined_regions_interpolated_with_all_MLD = pkl.load(open('combined_regions_interpolated_with_all_MLD.pkl', 'rb'))


In [9]:
# Choose MLD to use for classification between the 3 algorithms

# MLD = combined_regions_interpolated_with_all_MLD["tempMLD"]
# MLD_tag = 'tempMLD'
# MLD = combined_regions_interpolated_with_all_MLD["salinityMLD"]
# MLD_tag = 'salinityMLD'
MLD = combined_regions_interpolated_with_all_MLD["densityMLD"]
MLD_tag = 'densityMLD'

# Vertical structure of absolute salinity, to use for classification
data = combined_regions_interpolated_with_all_MLD.salinity_abs


In [10]:
def HYCOM_inc_dec_BL(data, MLD_T, MLD_PD, ML_delta, mode_tag):
    # Initialize lists for masks
    mask_incr = []
    mask_decr = []
    mask_inc_BL = []
    mask_dec_BL = []
    mask_inc_noBL = []
    mask_dec_noBL = []
    mask_all_BL = []
    mask_all_noBL = []
    
    negative_MLD_count = 0

    for i_event in np.arange(data.shape[0]):  # loop across events
        # Check if MLD values are negative
        if any(ml_d < 0 for ml_d in [MLD_T[i_event][index_time_start], MLD_PD[i_event][index_time_start]]):
            # If MLD is negative, set all masks for this event to False
            mask_incr.append(0)
            mask_decr.append(0)
            mask_inc_BL.append(0)
            mask_dec_BL.append(0)
            mask_inc_noBL.append(0)
            mask_dec_noBL.append(0)
            mask_all_BL.append(0)
            mask_all_noBL.append(0)
            
            # Increment the negative MLD count
            negative_MLD_count += 1
            
        else:
            # MIXED LAYER
            # Find depth index to insert ML
            i_depth_MLD = np.searchsorted(data[i_event].depth.values, MLD[i_event][index_time_start])
            # Create new depth axis which includes MLD
            depth_MLD = np.insert(data[i_event].depth.values, i_depth_MLD, MLD[i_event][index_time_start])
            # Calculate salinity at MLD via interpolation
            salinity_abs_upper = data[i_event][index_time_start].interp(depth=depth_MLD)

            # Calculate weighted average of salinity within the ML (from surface to MLD)
            depth_MLD_upper = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][index_time_start])
            depth_MLD_upper = np.insert(depth_MLD_upper, 0, 0)

            dz = (depth_MLD_upper[2:i_depth_MLD+3] - depth_MLD_upper[0:i_depth_MLD+1]) / 2
            MLS_abs_wgt = (np.sum(dz * salinity_abs_upper[0:i_depth_MLD+1])) / np.sum(dz)

            # LAYER BELOW ML
            if mode_tag == 'fraction':
                bottom_limit = MLD[i_event][index_time_start] + MLD[i_event][index_time_start] * ML_delta
            elif mode_tag == 'absolute':
                bottom_limit = MLD[i_event][index_time_start] + ML_delta
            depth_below_ML = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][index_time_start])
            i_depth_meters_below_MLD = np.searchsorted(depth_below_ML, bottom_limit)
            depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)
            depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)
            salinity_abs_lower = data[i_event][index_time_start].interp(depth=depth_below_ML)

            dz = (depth_below_ML[i_depth_MLD+2:i_depth_meters_below_MLD+2] - depth_below_ML[i_depth_MLD:i_depth_meters_below_MLD])/2
            salinity_abs_wgt_lower = (np.sum(dz * salinity_abs_lower[i_depth_MLD+1:i_depth_meters_below_MLD+1])) / np.sum(dz)

            mean_salinity_diff_below = salinity_abs_wgt_lower - MLS_abs_wgt

            # Calculate the difference between temperature-based MLD and density-based MLD
            MLD_diff = MLD_T[i_event][index_time_start] - MLD_PD[i_event][index_time_start]

            # Check if the difference is greater than 5 (like the distance between eachlayer)
            BL_yes_or_no = MLD_diff > 5

            # Append the result to the appropriate mask lists            
            if BL_yes_or_no:
                mask_all_BL.append(1)
                mask_all_noBL.append(0)
                mask_dec_noBL.append(0)
                mask_inc_noBL.append(0)
                if mean_salinity_diff_below > 0:
                    mask_incr.append(1)
                    mask_inc_BL.append(1)
                    mask_decr.append(0)
                    mask_dec_BL.append(0)
                elif mean_salinity_diff_below < 0:
                    mask_incr.append(0)
                    mask_inc_BL.append(0)
                    mask_decr.append(1)
                    mask_dec_BL.append(1)
                else:
                    mask_incr.append(0)
                    mask_decr.append(0)
                    mask_dec_BL.append(0)
                    mask_inc_BL.append(0)
            else:
                mask_all_noBL.append(1)
                mask_all_BL.append(0)
                mask_dec_BL.append(0)
                mask_inc_BL.append(0)
                if mean_salinity_diff_below > 0:
                    mask_incr.append(1)
                    mask_inc_noBL.append(1)
                    mask_decr.append(0)
                    mask_dec_noBL.append(0)
                elif mean_salinity_diff_below < 0:
                    mask_incr.append(0)
                    mask_inc_noBL.append(0)
                    mask_decr.append(1)
                    mask_dec_noBL.append(1)
                else:
                    mask_incr.append(0)
                    mask_decr.append(0)
                    mask_dec_noBL.append(0)
                    mask_inc_noBL.append(0)

    # Combine the masks
    combined_mask = {
        "incr": mask_incr,
        "decr": mask_decr,
        "inc_BL": mask_inc_BL,
        "dec_BL": mask_dec_BL,
        "inc_noBL": mask_inc_noBL,
        "dec_noBL": mask_dec_noBL,
        "all_BL": mask_all_BL,
        "all_noBL": mask_all_noBL
    }
    
    # Print the count of cases with negative MLD
    print(f"Number of cases with negative MLD @ -2 days: {negative_MLD_count}")
    return combined_mask

######################################################################################################

MLD_T = combined_regions_interpolated_with_all_MLD["tempMLD"]

MLD_PD = combined_regions_interpolated_with_all_MLD["densityMLD"]

combined_mask = HYCOM_inc_dec_BL(data, MLD_T, MLD_PD, ML_delta, mode_tag)
# combined_mask.keys()

mask_incr, mask_decr, mask_inc_BL, mask_dec_BL, mask_inc_noBL, mask_dec_noBL, mask_all_BL, mask_all_noBL = (np.ravel(combined_mask[key]).astype(bool) for key in combined_mask.keys())

# # Create a Dataset to hold all the masks
# mask_dataset = xr.Dataset({
#     "incr": xr.DataArray(mask_incr, dims='event'),
#     "decr": xr.DataArray(mask_decr, dims='event'),
#     "inc_BL": xr.DataArray(mask_inc_BL, dims='event'),
#     "dec_BL": xr.DataArray(mask_dec_BL, dims='event'),
#     "inc_noBL": xr.DataArray(mask_inc_noBL, dims='event'),
#     "dec_noBL": xr.DataArray(mask_dec_noBL, dims='event'),
#     "all_BL": xr.DataArray(mask_all_BL, dims='event'),
#     "all_noBL": xr.DataArray(mask_all_noBL, dims='event'),
# })

# # Save the Dataset to a NetCDF file
# mask_dataset.to_netcdf(f'allmasks_HYCOM_{MLD_tag}.nc')


Number of cases with negative MLD @ -2 days: 11


In [15]:
# List variable names
data = ['temperature_abs_all', 'salinity_abs_all', 'density_abs_all', \
        'temperature_abs_minus2_all', 'salinity_abs_minus2_all', 'density_abs_minus2_all', \
        'temperature_anom_minus2_all', 'salinity_anom_minus2_all', 'density_anom_minus2_all']

# Cases for selection
# data_mask = ['mask_incr',
#              'mask_decr']
data_mask = ['mask_incr', 'mask_decr', 'mask_inc_BL', 'mask_dec_BL', \
             'mask_inc_noBL', 'mask_dec_noBL', 'mask_all_BL', 'mask_all_noBL']


In [16]:
# Extract variables from combined_regions_interpolated_with_all_MLD
# Save only from -2 days to +14 days
temperature_abs_all = combined_regions_interpolated_with_all_MLD.temperature_abs[:,index_time_start:index_time_end+1,:].values
salinity_abs_all =  combined_regions_interpolated_with_all_MLD.salinity_abs[:,index_time_start:index_time_end+1,:].values
density_abs_all =  combined_regions_interpolated_with_all_MLD.density_abs[:,index_time_start:index_time_end+1,:].values
temperature_abs_minus2_all =  combined_regions_interpolated_with_all_MLD.temperature_abs_minus2[:,index_time_start:index_time_end+1,:].values
salinity_abs_minus2_all =  combined_regions_interpolated_with_all_MLD.salinity_abs_minus2[:,index_time_start:index_time_end+1,:].values
density_abs_minus2_all =  combined_regions_interpolated_with_all_MLD.density_abs_minus2[:,index_time_start:index_time_end+1,:].values
temperature_anom_minus2_all =  combined_regions_interpolated_with_all_MLD.temperature_anom_minus2[:,index_time_start:index_time_end+1,:].values
salinity_anom_minus2_all = combined_regions_interpolated_with_all_MLD.salinity_anom_minus2[:,index_time_start:index_time_end+1,:].values
density_anom_minus2_all = combined_regions_interpolated_with_all_MLD.density_anom_minus2[:,index_time_start:index_time_end+1,:].values

longitudes_2d_map  = combined_regions_interpolated_with_all_MLD.longitude.values
latitudes_2d_map  = combined_regions_interpolated_with_all_MLD.latitude.values


In [17]:
# Function to plot single events
def plot_single_events(ax_handle, xr_D, var_name, xaxis, flag_xaxis_label=True, cmap='bwr', norm_flag=True, vmin=None, vmax=None, levels=None, yaxis=None):
    fontsize = 18

    # Ensure that vmin is less than vmax
    if vmin is None:
        vmin_value = np.min(xr_D)
    else:
        vmin_value = vmin

    if vmax is None:
        vmax_value = np.max(xr_D)
    else:
        vmax_value = vmax

    # If vmin is greater than or equal to vmax, sort the values
    if vmin_value >= vmax_value:
        sorted_values = np.sort(xr_D.values.flatten())
        vmin_value = sorted_values[0]
        vmax_value = sorted_values[-1]

    # Ensure vcenter is in ascending order
    vcenter_value = 0.5 * (vmin_value + vmax_value)

    if vcenter_value >= vmax_value:
        vcenter_value = vmin_value + 0.5 * (vmax_value - vmin_value)

    if levels is not None:
        # Check if levels is an integer
        if isinstance(levels, int):
            levels = np.linspace(vmin_value, vmax_value, levels)

    if norm_flag:
        norm = mcolors.TwoSlopeNorm(vmin=vmin_value, vmax=vmax_value, vcenter=vcenter_value)
        im = ax_handle.contourf(xaxis, combined_regions_interpolated_with_all_MLD.depth[:], xr_D.transpose(), cmap=cmap, levels=levels, norm=norm, vmin=vmin_value, vmax=vmax_value)
    else:
        im = ax_handle.contourf(xaxis, combined_regions_interpolated_with_all_MLD.depth[:], xr_D.transpose(), cmap=cmap, levels=levels, vmin=vmin_value, vmax=vmax_value)

    ax_handle.set_ylabel('Depth (m)')
    ax_handle.axvline(0, c='k')

    divider = make_axes_locatable(ax_handle)
    cax = divider.append_axes('right', size='4%', pad=0.01)

    # Check if levels is not None before using len
    if levels is not None:
        cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
    else:
        cb = plt.colorbar(im, cax=cax, orientation='vertical')

    cax.tick_params(labelsize=8)
    ax_handle.invert_yaxis()

In [40]:
from itertools import islice

# Define the number of events
num_events = combined_regions_interpolated_with_all_MLD.index

ihatch_factor = 1.96

min_value_anom_s = -2.2
max_value_anom_s = 2.2
min_value_anom_t = -12
max_value_anom_t = 12
min_value_anom_pd = -3
max_value_anom_pd = 3

# min_value_anom_s = np.nanmin(salinity_anom_minus2_all) #-1.5
# max_value_anom_s = np.nanmax(salinity_anom_minus2_all) #1.5
# min_value_anom_t = np.nanmin(temperature_anom_minus2_all) #-2
# max_value_anom_t = np.nanmax(temperature_anom_minus2_all) #2
# min_value_anom_pd = np.nanmin(density_anom_minus2_all) #-1
# max_value_anom_pd = np.nanmax(density_anom_minus2_all) #1

min_value_abs_s = np.nanmin(salinity_abs_all) #31
max_value_abs_s = np.nanmax(salinity_abs_all) #37
min_value_abs_t = np.nanmin(temperature_abs_all) #-2
max_value_abs_t = np.nanmax(temperature_abs_all) #32
min_value_abs_pd = np.nanmin(density_abs_all) #1019
max_value_abs_pd = np.nanmax(density_abs_all) #1027

num_levels_anom = 51
num_levels_abs = 51
# Define the maximum depth to display (e.g., 200 meters)
max_depth = 200
meters_belowMLD = 50
    
# Loop through each event
for i in num_events:
# for i in islice(num_events, 27, None):
#     Create subplots with only left and middle columns
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(22, 14))   
    
    # Plot left column
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[0, 0].set_ylim([0, max_depth])
    else:
        axes[0, 0].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    axes[0, 0].plot(salinity_abs_all[i, 0, :], combined_regions_interpolated_with_all_MLD.depth, color='green', linewidth=1.5, )
    axes[0, 0].axhline(y=combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start], color='red', linestyle='--', label='Density MLD')
    axes[0, 0].axhline(y=combined_regions_interpolated_with_all_MLD.salinityMLD[i, index_time_start], color='green', linestyle='-.', label='Salinity MLD')
    axes[0, 0].axhline(y=combined_regions_interpolated_with_all_MLD.tempMLD[i, index_time_start], color='blue', linestyle=':', label='Temperature MLD')
    axes[0, 0].axhline(y=combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start] + meters_belowMLD, color='red', linestyle='solid', linewidth=.5)
    axes[0, 0].invert_yaxis()
    axes[0, 0].legend(loc='upper right')

    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[1, 0].set_ylim([0, max_depth])
    else:
        axes[1, 0].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD]) 
    axes[1, 0].plot(temperature_abs_all[i, 0, :], combined_regions_interpolated_with_all_MLD.depth, color='blue', linewidth=1.5)
    axes[1, 0].axhline(y=combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start], color='red', linestyle='--')
    axes[1, 0].axhline(y=combined_regions_interpolated_with_all_MLD.salinityMLD[i, index_time_start], color='green', linestyle='--')
    axes[1, 0].axhline(y=combined_regions_interpolated_with_all_MLD.tempMLD[i, index_time_start], color='blue', linestyle='--')
    axes[1, 0].invert_yaxis()
    
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[2, 0].set_ylim([0, max_depth])
    else:
        axes[2, 0].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD]) 
    axes[2, 0].plot(density_abs_all[i, 0, :], combined_regions_interpolated_with_all_MLD.depth, color='red', linewidth=1.5)
    axes[2, 0].axhline(y=combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start], color='red', linestyle='--')
    axes[2, 0].axhline(y=combined_regions_interpolated_with_all_MLD.salinityMLD[i, index_time_start], color='green', linestyle='--')
    axes[2, 0].axhline(y=combined_regions_interpolated_with_all_MLD.tempMLD[i, index_time_start], color='blue', linestyle='--')
    axes[2, 0].invert_yaxis()

    # Add titles to left panels
    axes[0, 0].set_title('salinity_abs @ -2 days', fontsize=14)
    axes[1, 0].set_title('temperature_abs @ -2 days', fontsize=14)
    axes[2, 0].set_title('density_abs @ -2 days ', fontsize=14)
    # Add axis labels to the left panels
    axes[0, 0].set_xlabel('Salinity')
    axes[1, 0].set_xlabel('Temperature')
    axes[2, 0].set_xlabel('Density')
    axes[0, 0].set_ylabel('Depth (m)')
    axes[1, 0].set_ylabel('Depth (m)')
    axes[2, 0].set_ylabel('Depth (m)')
    
    # Call the function for the middle panels
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[0, 1].set_ylim([0, max_depth])
    else:
        axes[0, 1].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])
    plot_single_events(ax_handle=axes[0, 1], xr_D=salinity_abs_all[i, :, :], var_name='Salinity', xaxis=time_plot, cmap='viridis', vmin=min_value_abs_s, vmax=max_value_abs_s, levels=num_levels_abs)
    axes[0, 1].plot(time_plot, combined_regions_interpolated_with_all_MLD.salinityMLD[i, index_time_start:index_time_end + 1].values, linewidth=2, color='r', label='Salinity MLD')
    axes[0, 1].set_title('salinity_abs', fontsize=14)
    axes[0, 1].set_xlabel('days')
    
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[1, 1].set_ylim([0, max_depth])
    else:
        axes[1, 1].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    plot_single_events(ax_handle=axes[1, 1], xr_D=temperature_abs_all[i, :, :], var_name='Temperature', xaxis=time_plot, cmap='viridis', vmin=min_value_abs_t, vmax=max_value_abs_t, levels=num_levels_abs)
    axes[1, 1].plot(time_plot, combined_regions_interpolated_with_all_MLD.tempMLD[i, index_time_start:index_time_end + 1].values, linewidth=2, color='r', label='Temperature MLD')
    axes[1, 1].set_title('temperature_abs', fontsize=14)
    axes[1, 1].set_xlabel('days')
    
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[2, 1].set_ylim([0, max_depth])
    else:
        axes[2, 1].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    plot_single_events(ax_handle=axes[2, 1], xr_D=density_abs_all[i, :, :], var_name='Density', xaxis=time_plot, cmap='viridis', vmin=min_value_abs_pd, vmax=max_value_abs_pd, levels=num_levels_abs)
    axes[2, 1].plot(time_plot, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start:index_time_end + 1].values, linewidth=2, color='r', label='Density MLD')
    axes[2, 1].set_title('density_abs', fontsize=14)
    axes[2, 1].set_xlabel('days')

    # Add legends to the plots
    axes[0, 1].legend(loc='lower right')
    axes[1, 1].legend(loc='lower right')
    axes[2, 1].legend(loc='lower right')


    # Call the function for right panels
    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[0, 2].set_ylim([0, max_depth])
    else:
        axes[0, 2].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    plot_single_events(ax_handle=axes[0, 2], xr_D=salinity_anom_minus2_all[i, :, :], var_name='Salinity Anomaly', xaxis=time_plot, vmin=min_value_anom_s, vmax=max_value_anom_s, levels=num_levels_anom)
    axes[0, 2].plot(time_plot, combined_regions_interpolated_with_all_MLD.densityMLD[i,index_time_start:index_time_end+1].values, linewidth=2, color='k', label='Density MLD')
    axes[0, 2].set_title('salinity_anom minus @ 2 days', fontsize=14)
    axes[0, 2].set_xlabel('days')

    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[1, 2].set_ylim([0, max_depth])
    else:
        axes[1, 2].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    plot_single_events(ax_handle=axes[1, 2], xr_D=temperature_anom_minus2_all[i, :, :], var_name='Temperature Anomaly', xaxis=time_plot, vmin=min_value_anom_t, vmax=max_value_anom_t, levels=num_levels_anom)
    axes[1, 2].plot(time_plot, combined_regions_interpolated_with_all_MLD.densityMLD[i,index_time_start:index_time_end+1].values, linewidth=2, color='k', label='Density MLD')
    axes[1, 2].set_title('temperature_anom minus @ 2 days', fontsize=14)
    axes[1, 2].set_xlabel('days')

    if combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD<max_depth:
        axes[2, 2].set_ylim([0, max_depth])
    else:
        axes[2, 2].set_ylim([0, combined_regions_interpolated_with_all_MLD.densityMLD[i, index_time_start]+2*meters_belowMLD])    
    plot_single_events(ax_handle=axes[2, 2], xr_D=density_anom_minus2_all[i, :, :], var_name='Density Anomaly', xaxis=time_plot, vmin=min_value_anom_pd, vmax=max_value_anom_pd, levels=num_levels_anom)
    axes[2, 2].plot(time_plot, combined_regions_interpolated_with_all_MLD.densityMLD[i,index_time_start:index_time_end+1].values, linewidth=2, color='k', label='Density MLD')
    axes[2, 2].set_title('density_anom minus @ 2 days', fontsize=14)
    axes[2, 2].set_xlabel('days')

    # Add legends to the plots
    axes[0, 2].legend(loc='lower right')
    axes[1, 2].legend(loc='lower right')
    axes[2, 2].legend(loc='lower right')
    
    # Add the mask values outside the subplot
    for j, mask_name in enumerate(data_mask):
        fig.text(.95, 0.85 - 0.1 * j, f'{mask_name}: {globals()[mask_name][i]}', ha='center', va='center', fontsize=16)
    
    # Determine title color based on conditions
    title_color = 'black'  # Default color
    if mask_incr[i]:
        title_color = 'red'
        title_suffix = 'BL' if mask_inc_BL[i] else 'noBL'
    elif mask_decr[i]:
        title_color = 'blue'
        title_suffix = 'BL' if mask_inc_BL[i] else 'noBL'

    fig.suptitle(f'Event #{i.values}, {title_suffix}', fontsize=26, y=0.985, x=0.1, color=title_color)

    
    # Adjust layout and show plot
    plt.tight_layout(rect=[0, 0, 0.9, 1])
#     ciao
    plt.savefig(f'single_event_plots_MLD_masks/event_{i.values}_{MLD_tag}.jpg', dpi=1000)
    plt.close()

#     plt.show()


<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

...

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 'boundaries' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(im, cax=cax, orientation='vertical', boundaries=levels)
<ipython-input-17-b341386dca0f>:47: MatplotlibDeprecationWarning: The 

In [ ]:
# i_event = 906
# i_time = 161 # corresponds to timedelta64 17:00:00 after the event
# pressure = gsw.p_from_z(-combined_regions_interpolated.depth.values, float(combined_regions_interpolated.latitude.values[0]))
# conservative_temperature_nonans = combined_regions_interpolated.temperature_abs.values[i_event, i_time][~np.isnan(combined_regions_interpolated["temperature_abs"].values[i_event, i_time])]
# absolute_salinity_nonans = combined_regions_interpolated.salinity_abs.values[i_event, i_time][~np.isnan(combined_regions_interpolated["salinity_abs"].values[i_event, i_time])]
# potential_density_nonans = combined_regions_interpolated.density_abs.values[i_event, i_time][~np.isnan(combined_regions_interpolated["density_abs"].values[i_event, i_time])]

# h = HolteAndTalley(pressures=pressure[:conservative_temperature_nonans.shape[0]],
#                    temperatures=conservative_temperature_nonans,
#                    salinities=absolute_salinity_nonans,
#                    densities=potential_density_nonans)

# pressure[:conservative_temperature_nonans.shape[0]]

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming your variable is named 'my_variable'
# # Replace 'my_variable' with the actual name of your variable
# i = 1132
# # Generate sample data (replace this with your actual data)
# my_variable = salinity_abs_all[i, :, 0:40]

# # Create a time array (replace this with your actual time array)
# time_array = time_plot

# # Create a depth array (replace this with your actual depth array)
# depth_array = np.arange(0, 40)

# # Create a meshgrid for time and depth
# time_plot_test, depth_plot = np.meshgrid(time_array, depth_array)

# # Plot the variable using a heatmap or contour plot
# plt.figure(figsize=(10, 6))
# plt.pcolormesh(time_plot_test, depth_plot, my_variable.T, cmap='viridis')
# plt.gca().invert_yaxis()
# plt.colorbar(label='Variable')
# plt.xlabel('Time')
# plt.ylabel('Depth')
# plt.title('Time vs. Depth Plot')
# plt.show()


In [13]:
# def HYCOM_inc_dec_BL(data, MLD_T, MLD_PD, ML_delta, mode_tag):
#     # Initialize lists for masks
#     mask_incr = []
#     mask_decr = []
#     mask_inc_BL = []
#     mask_dec_BL = []
#     mask_inc_noBL = []
#     mask_dec_noBL = []
#     mask_all_BL = []
#     mask_all_noBL = []

#     for i_event in np.arange(data.shape[0]):  # loop across events

#         # MIXED LAYER
#         # Find depth index to insert ML
#         i_depth_MLD = np.searchsorted(data[i_event].depth.values, MLD[i_event][index_time_start])
#         # Create new depth axis which includes MLD
#         depth_MLD = np.insert(data[i_event].depth.values, i_depth_MLD, MLD[i_event][index_time_start])
#         # Calculate salinity at MLD via interpolation
#         salinity_abs_upper = data[i_event][index_time_start].interp(depth=depth_MLD)

#         # Calculate weighted average of salinity within the ML (from surface to MLD)
#         depth_MLD_upper = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][index_time_start])
#         depth_MLD_upper = np.insert(depth_MLD_upper, 0, 0)

#         dz = (depth_MLD_upper[2:i_depth_MLD+3] - depth_MLD_upper[0:i_depth_MLD+1]) / 2
#         MLS_abs_wgt = (np.sum(dz * salinity_abs_upper[0:i_depth_MLD+1])) / np.sum(dz)

#         # LAYER BELOW ML
#         if mode_tag == 'fraction':
#             bottom_limit = MLD[i_event][index_time_start] + MLD[i_event][index_time_start] * ML_delta
#         elif mode_tag == 'absolute':
#             bottom_limit = MLD[i_event][index_time_start] + ML_delta
#         depth_below_ML = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][index_time_start])
#         i_depth_meters_below_MLD = np.searchsorted(depth_below_ML, bottom_limit)
#         depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)
#         depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)
#         salinity_abs_lower = data[i_event][index_time_start].interp(depth=depth_below_ML)

#         dz = (depth_below_ML[i_depth_MLD+2:i_depth_meters_below_MLD+2] - depth_below_ML[i_depth_MLD:i_depth_meters_below_MLD])/2
#         salinity_abs_wgt_lower = (np.sum(dz * salinity_abs_lower[i_depth_MLD+1:i_depth_meters_below_MLD+1])) / np.sum(dz)

#         mean_salinity_diff_below = salinity_abs_wgt_lower - MLS_abs_wgt

#         # Calculate the difference between temperature-based MLD and density-based MLD
#         MLD_diff = MLD_T[i_event][index_time_start] - MLD_PD[i_event][index_time_start]

#         # Check if the difference is greater than 5 (like the distance between eachlayer)
#         BL_yes_or_no = MLD_diff > 5

#         # Append the result to the appropriate mask lists            
#         if BL_yes_or_no:
#             mask_all_BL.append(1)
#             mask_all_noBL.append(0)
#             mask_dec_noBL.append(0)
#             mask_inc_noBL.append(0)
#             if mean_salinity_diff_below > 0:
#                 mask_incr.append(1)
#                 mask_inc_BL.append(1)
#                 mask_decr.append(0)
#                 mask_dec_BL.append(0)
#             elif mean_salinity_diff_below < 0:
#                 mask_incr.append(0)
#                 mask_inc_BL.append(0)
#                 mask_decr.append(1)
#                 mask_dec_BL.append(1)
#             else:
#                 mask_incr.append(0)
#                 mask_decr.append(0)
#                 mask_dec_BL.append(0)
#                 mask_inc_BL.append(0)
#         else:
#             mask_all_noBL.append(1)
#             mask_all_BL.append(0)
#             mask_dec_BL.append(0)
#             mask_inc_BL.append(0)
#             if mean_salinity_diff_below > 0:
#                 mask_incr.append(1)
#                 mask_inc_noBL.append(1)
#                 mask_decr.append(0)
#                 mask_dec_noBL.append(0)
#             elif mean_salinity_diff_below < 0:
#                 mask_incr.append(0)
#                 mask_inc_noBL.append(0)
#                 mask_decr.append(1)
#                 mask_dec_noBL.append(1)
#             else:
#                 mask_incr.append(0)
#                 mask_decr.append(0)
#                 mask_dec_noBL.append(0)
#                 mask_inc_noBL.append(0)

#     # Combine the masks
#     combined_mask = {
#         "incr": mask_incr,
#         "decr": mask_decr,
#         "inc_BL": mask_inc_BL,
#         "dec_BL": mask_dec_BL,
#         "inc_noBL": mask_inc_noBL,
#         "dec_noBL": mask_dec_noBL,
#         "all_BL": mask_all_BL,
#         "all_noBL": mask_all_noBL
#     }

#     return combined_mask


# ######################################################################################################

# MLD_T = combined_regions_interpolated_with_all_MLD["tempMLD"]

# MLD_PD = combined_regions_interpolated_with_all_MLD["densityMLD"]


# combined_mask = HYCOM_inc_dec_BL(data, MLD_T, MLD_PD, ML_delta, mode_tag)
# # combined_mask.keys()


# mask_incr, mask_decr, mask_inc_BL, mask_dec_BL, mask_inc_noBL, mask_dec_noBL, mask_all_BL, mask_all_noBL = (np.ravel(combined_mask[key]).astype(bool) for key in combined_mask.keys())


# # # Create a Dataset to hold all the masks
# # mask_dataset = xr.Dataset({
# #     "incr": xr.DataArray(mask_incr, dims='event'),
# #     "decr": xr.DataArray(mask_decr, dims='event'),
# #     "inc_BL": xr.DataArray(mask_inc_BL, dims='event'),
# #     "dec_BL": xr.DataArray(mask_dec_BL, dims='event'),
# #     "inc_noBL": xr.DataArray(mask_inc_noBL, dims='event'),
# #     "dec_noBL": xr.DataArray(mask_dec_noBL, dims='event'),
# #     "all_BL": xr.DataArray(mask_all_BL, dims='event'),
# #     "all_noBL": xr.DataArray(mask_all_noBL, dims='event'),
# # })

# # # Save the Dataset to a NetCDF file
# # mask_dataset.to_netcdf(f'allmasks_HYCOM_{MLD_tag}.nc')